In [1]:
import torch
import torchvision
import torchvision.transforms.v2 as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cnn import *
from ffn import *

In [2]:
OUTPUT_DIM = 10
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

IMAGE_HEIGHT = 28
IMAGE_WIDTH = 28
INPUT_DIM = IMAGE_HEIGHT * IMAGE_WIDTH

In [3]:
transform = transforms.Compose([                            # Use transforms to convert images to tensors and normalize them
    transforms.ToTensor(),                                  # convert images to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])             # Common method for grayscale images
])

batch_size = 64 # 32–128 is typical

/Users/griffinmichalak/Desktop/code/fall25/intro-ai/PA3-1/venv/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
trainset = torchvision.datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = torchvision.datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)

In [5]:
# input = torch.Tensor(batch_size, INPUT_DIM, IMAGE_HEIGHT, IMAGE_WIDTH)

# feedforward_net = FF_Net()
conv_net = Conv_Net()

In [6]:
criterion = nn.CrossEntropyLoss() # loss function

# optimizer_ffn = optim.Adam([], lr=LEARNING_RATE)
optimizer_cnn = optim.Adam(conv_net.parameters(), lr=LEARNING_RATE)

### Training

In [ ]:
num_epochs_cnn = NUM_EPOCHS

for epoch in range(num_epochs_cnn):  # loop over the dataset multiple times
    running_loss_cnn = 0.0

    correct = 0

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer_cnn.zero_grad()

        # forward + backward + optimize
        outputs = conv_net(inputs) # forward pass happens here
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss_cnn += loss.item()

    print(f"Training loss: {running_loss_cnn}")

print('Finished Training')

torch.save(conv_net.state_dict(), 'cnn.pth')  # Saves model file (upload with submission)

Training loss: 471.73059195280075
Training loss: 283.654587469995
Training loss: 238.80991011857986
Training loss: 210.82315225154161
Training loss: 187.6154305972159
Finished Training


# Evaluation

In [8]:
correct_ffn = 0
total_ffn = 0

correct_cnn = 0
total_cnn = 0

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = conv_net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_cnn = correct_cnn + 1
            total_cnn = total_cnn + 1


# print('Accuracy for feedforward network: ', correct_ffn/total_ffn)
print('Accuracy for convolutional network: ', correct_cnn/total_cnn)

Accuracy for convolutional network:  0.9119
